# Vanilla FL Implementation

1. Install necessary dependencies

In [10]:
pip install torch numpy torchvision

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
    --------------------------------------- 0.0/1.6 MB 330.3 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.6 MB 737.3 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.6 MB 1.4 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.6 MB 1.8 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.6 MB 1.9 MB/s eta 0:00:01
   ---------------- ----------------------- 0.7/1.6 MB 2.1 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.6 MB 2.3 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.6 MB 2.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.1/1.6 MB 2.4 MB/s eta 0:00:01
   ---------------------------------

2. Use PyTorch to define the model that will be distributed to the clients

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Example: Simple Neural Network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)  # Output for 10 classes

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten input
        x = self.relu(self.fc1(x))
        return self.fc2(x)


3. Each client will train the model locally on their dataset

In [11]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Load and partition dataset (e.g., MNIST)
def load_client_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)

    # Split dataset into parts for clients
    client_data = random_split(dataset, [6000] * 10)  # 10 clients with ~6000 samples each
    return client_data


4. Each client trains the model locally using their dataset

In [12]:
def train_client(model, data_loader, epochs=1, lr=0.01):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(epochs):
        for inputs, labels in data_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Return the trained model parameters
    return {name: param.data.clone() for name, param in model.state_dict().items()}


5. The central server aggregates updates from all clients.

In [ ]:
def federated_averaging(client_updates):
    avg_params = {}
    num_clients = len(client_updates)

    # Average parameters across clients
    for name in client_updates[0]:
        avg_params[name] = sum([client[name] for client in client_updates]) / num_clients

    return avg_params


6. Coordinate training between the server and clients.

In [ ]:
def vanilla_federated_learning(num_rounds=5, num_clients=10, epochs=1):
    # Initialize global model
    global_model = SimpleNN()
    client_data = load_client_data()

    for round in range(num_rounds):
        print(f"Round {round + 1}/{num_rounds}")
        client_updates = []

        # Each client trains the model on local data
        for client_id in range(num_clients):
            data_loader = DataLoader(client_data[client_id], batch_size=32, shuffle=True)
            client_model = SimpleNN()  # Clone the global model
            client_model.load_state_dict(global_model.state_dict())  # Sync with the global model
            client_update = train_client(client_model, data_loader, epochs=epochs)
            client_updates.append(client_update)

        # Server aggregates updates
        avg_params = federated_averaging(client_updates)

        # Update global model
        global_model.load_state_dict(avg_params)

    print("Federated Learning Completed")
    return global_model


7. After training, evaluate the global model on a test dataset.

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")


8. Run the experiment

In [ ]:
if __name__ == "__main__":
    global_model = vanilla_federated_learning()

    # Load test data
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Evaluate the global model
    evaluate_model(global_model, test_loader)
